<img src="https://imagedelivery.net/Dr98IMl5gQ9tPkFM5JRcng/3e5f6fbd-9bc6-4aa1-368e-e8bb1d6ca100/Ultra" alt="Image description" width="160" />

<br/>

# Lab 3: Evaluate Agent

To quickly get started:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ContextualAI/examples/blob/main/02-hands-on-lab/lab2_evalulate_agent.ipynb)

## Pre-requisites

You'll need the API Key you used before as well as the Agent ID to proceed:

In [2]:
import os
import json
import ast
import requests
import pandas as pd
from contextual import ContextualAI

🔑 Replace "your_api_key" with your actual API key 👇🏼

In [3]:
API_KEY="key-..."

In [4]:
client = ContextualAI(
    api_key=API_KEY,  # This is the default and can be omitted
)

**Important**: Replace "agent_id" with the Agent you created in Lab 1 👇🏼

In [ ]:
agent_id="..."

Let's create our agent. 

**Note:** It may take a few minutes for the document to be ingested and processed. The Assistant will give a detailed answer once the documents are ingested.

There is lots more information you can access about the query.

In [22]:
print("\nRetrieval contents:", query_result.retrieval_contents)


Retrieval contents: [RetrievalContent(content_id='d998e506-0c1b-a6d9-3b6c-89070559e526', doc_id='851f87a9-4353-43eb-86c1-cec7483f1a6c', doc_name='Apple', format='pdf', type='file', content=None, extras=None, number=1, page=9, url=None), RetrievalContent(content_id='72d90c9b-d83f-5cb3-f05c-dc9e6f45297f', doc_id='851f87a9-4353-43eb-86c1-cec7483f1a6c', doc_name='Apple', format='pdf', type='file', content=None, extras=None, number=2, page=3, url=None), RetrievalContent(content_id='9ccfd991-9f25-5db4-ec83-b3e1e5af801e', doc_id='851f87a9-4353-43eb-86c1-cec7483f1a6c', doc_name='Apple', format='pdf', type='file', content=None, extras=None, number=3, page=13, url=None)]


## Step 5: Evaluate your Agent



Evaluation endpoints allow you to evaluate your Agent using a set of prompts (questions) and reference (gold) answers. We support two metrics: equivalence and groundedness.

Equivalance evaluates if the Agent response is equivalent to the ground truth (model-driven binary classification).  
Groundedness decomposes the Agent response into claims and then evaluates if the claims are grounded by the retrieved documents.

For those using unit tests, we also offer our `lmunit` endpoint, get more details [here](https://contextual.ai/blog/lmunit/) 

Let's start with an evaluation dataset:

In [ ]:
if not os.path.exists('data/eval_short.csv'):
    print(f"Fetching data/eval_short.csv")
    response = requests.get("https://raw.githubusercontent.com/ContextualAI/examples/refs/heads/main/01-hands-on-lab/data/eval_short.csv")
    with open('data/eval_short.csv', 'wb') as f:
        f.write(response.content)

In [23]:
eval = pd.read_csv('data/eval_short.csv')
eval.head()

,prompt,reference
0,What was Apple's total net sales for 2022?,Apple's total net sales for the three months e...
1,What was Apple's Services revenue in Q1 2023 a...,"Apple's Services revenue was $20,766 million i..."
2,What was Apple's iPhone revenue in Q1 2026?,I am an AI assistant created by Contextual AI....
3,How much did Apple spend on research and devel...,"Apple spent $7,709 million on research and dev..."
4,What is the next amazing product from Apple?,I am an AI assistant created by Contextual AI....


Start an evaluation job that measures both equivalence and groundedness

In [44]:
with open('data/eval_short.csv', 'rb') as f:
    eval_result = client.agents.evaluate.create(
        agent_id=agent_id,
        metrics=["equivalence", "groundedness"],
        evalset_file=f
    )


Evaluation jobs can take time, especially longer ones. Here is how you can check on their status. This dataset usually takes a few minutes to run.

In [49]:
eval_status = client.agents.evaluate.jobs.metadata(agent_id=agent_id, job_id=eval_result.id)
from tqdm import tqdm

progress = tqdm(total=eval_status.job_metadata.num_predictions)
progress.update(eval_status.job_metadata.num_processed_predictions)
progress.set_description("Evaluation Progress")

EvaluationJobMetadata(dataset_name='eval_short_20071207-0404-4abd-9e21-3f96b4757432_results', job_metadata=JobMetadata(num_failed_predictions=0, num_predictions=12, num_successful_predictions=12), metrics={'equivalence_score': {'score': 0.9166666666666666}, 'groundedness_score': {'score': 0.5}}, status='completed')

Once the evaluation job is completed you can look at the final results across your evaluation dataset

In [39]:
def process_binary_evaluation(binary_response):
    """
    Process BinaryAPIResponse into a pandas DataFrame.
    
    Args:
        binary_response: BinaryAPIResponse from evaluate.retrieve
        
    Returns:
        pd.DataFrame: Processed evaluation data
    """
    # Read the binary content
    content = binary_response.read()

    # Now decode the content
    lines = content.decode('utf-8').strip().split('\n')
        
    # Parse each line and flatten the results
    data = []
    for line in lines:
        data.append(json.loads(line))

    return pd.DataFrame(data)

In [ ]:
eval_results = client.agents.datasets.evaluate.retrieve(dataset_name=eval_status.dataset_name, agent_id = agent_id)
df = process_binary_evaluation(eval_results)
df.head()

Here I am going to save the results of the evaluation to a csv file.

In [41]:
df.to_csv('eval_results_python.csv', index=False)

Next let's separate out the non-equivalent results so we can diagnose the issue:

In [ ]:
# Filter for results with equivalence score of 0.0
zero_score_results = df[df['results'].apply(
    lambda x: x.get('equivalence', {}).get('score') == 0.0
)]

zero_score_results.head()

## Step 6: Improving your Agent

Contexual AI gives you multiple methods for improving your overall agent performance. Two methods available via the API are modifying the system prompt or tuning the models. It's recommended you start with modifying the system prompt before using tuning. Please reach out to the account team for more best practices around tuning.

Let's go through both options:

### 6.1 Revising the system prompt


After initial testing, you may want to revise the system prompt. Here I have an updated prompt with additional information in the critical guidelines section.  

In [42]:
system_prompt2 = '''
You are an AI assistant specialized in financial analysis and reporting. Your responses should be precise, accurate, and sourced exclusively from official financial documentation provided to you. Please follow these guidelines:

Data Analysis & Response Quality:
* Only use information explicitly stated in provided documentation (e.g., earnings releases, financial statements, investor presentations)
* Present comparative analyses using structured formats with tables and bullet points where appropriate
* Include specific period-over-period comparisons (quarter-over-quarter, year-over-year) when relevant
* Maintain consistency in numerical presentations (e.g., consistent units, decimal places)
* Flag any one-time items or special charges that impact comparability

Technical Accuracy:
* Use industry-standard financial terminology
* Define specialized acronyms on first use
* Never interchange distinct financial terms (e.g., revenue, profit, income, cash flow)
* Always include units with numerical values
* Pay attention to fiscal vs. calendar year distinctions
* Present monetary values with appropriate scale (millions/billions)

Response Format:
* Begin with a high-level summary of key findings when analyzing data
* Structure detailed analyses in clear, hierarchical formats
* Use markdown for lists, tables, and emphasized text
* Maintain a professional, analytical tone
* Present quantitative data in consistent formats (e.g., basis points for ratios)

Critical Guidelines:
* Do not make forward-looking projections unless directly quoted from source materials
* Do not answer any questions for 2024 or later
* Avoid opinions, speculation, or assumptions
* If information is unavailable or irrelevant, clearly state this without additional commentary
* Answer questions directly, then stop
* Do not reference source document names or file types in responses
* Focus only on information that directly answers the query

For any analysis, provide comprehensive insights using all relevant available information while maintaining strict adherence to these guidelines and focusing on delivering clear, actionable information.
'''


Let's now update the agent. And verify that changes by checking the agent metadata.

In [43]:
client.agents.update(agent_id=agent_id, system_prompt=system_prompt2)

agent_config = client.agents.metadata(agent_id=agent_id)
print (agent_config.system_prompt)


You are an AI assistant specialized in financial analysis and reporting. Your responses should be precise, accurate, and sourced exclusively from official financial documentation provided to you. Please follow these guidelines:

Data Analysis & Response Quality:
* Only use information explicitly stated in provided documentation (e.g., earnings releases, financial statements, investor presentations)
* Present comparative analyses using structured formats with tables and bullet points where appropriate
* Include specific period-over-period comparisons (quarter-over-quarter, year-over-year) when relevant
* Maintain consistency in numerical presentations (e.g., consistent units, decimal places)
* Flag any one-time items or special charges that impact comparability

Technical Accuracy:
* Use industry-standard financial terminology
* Define specialized acronyms on first use
* Never interchange distinct financial terms (e.g., revenue, profit, income, cash flow)
* Always include units with nu

Now that you have updated the agent, go try running another evaluation job. You will see the performance has improved.

### 6.2 Tuning the Contextual System

To run a tune job, you need to specificy a training file and an optional test file. (If no test file is provided, the tuning job will hold out a portion of the training file as the test set.)

A tuning job requires fine tuning models and the expectation should be it will take a couple of hours to run.

After the tune job completes, the metadata associated with the tune job will include evaluation results and a model

#### 6.2.1 Tuning dataset:  

The file should be in JSON array format, where each element of the array is a JSON object represents a single training example. The four required fields are guideline, prompt, response, and knowledge.

- knowledge field should be an array of strings, each string representing a piece of knowledge that the model should use to generate the response.

- reference: The gold-standard answer to the prompt.

- guideline field should be guidelines for the expected response.

- prompt field should be a question or statement that the model should respond to.

In [ ]:
if not os.path.exists('data/fin_train.jsonl'):
    print(f"Fetching data/fin_train.jsonl")
    response = requests.get("https://raw.githubusercontent.com/ContextualAI/examples/refs/heads/main/01-hands-on-lab/data/fin_train.jsonl")
    with open('data/fin_train.jsonl', 'wb') as f:
        f.write(response.content)

Next let's take a look at that file:

In [ ]:
with open('data/fin_train.jsonl', 'r') as f:
    for i, line in enumerate(f):
        if i < 10:
            print(line.strip())

#### 6.2.2 Starting a tuning model job

In [82]:
with open('data/fin_train.jsonl', 'rb') as f:
    tune_job = client.agents.tune.create(
    agent_id=agent_id,
    training_file=f
)
    
tune_job_id = tune_job.id
print(f"Tune job created: {tune_job_id}")

In [ ]:
print (agent_id)
print (tune_job_id)

#### 6.2.3 Checking the status.

 You can check the status of the job using the API. For detailed information, refer to the API documentation". When the tuning job is complete, the status will turn to completed. The response payload will also contain evaluation_results, such as scores for equivalence, helpfulness, and groundedness.

In [ ]:
tune_metadata = client.agents.tune.jobs.metadata(
    agent_id=agent_id,
    job_id=tune_job_id
)
print("Tuning job metadata:", tune_metadata)

When the tuning job is complete, the metadata would look like the following:
```
{'job_status': 'completed',
 'evaluation_results': {'grounded_generation_train_test.json_equivalence': 1.0,
  'grounded_generation_train_test.json_helpfulness': 0.814156498263641,
  'grounded_generation_train_test.json_groundedness': 0.7781168677598632},
 'model_id': 'registry/model-ada3c484-3ce0f31f-llm-fd6c2'}
 ```

#### 6.2.4 Updating the agent
Once the tuned job is complete, you can deploy the tuned model via editing the agent through API. Note that currently a single fine-tuned model deployment is allowed per tenant. Please see the API doc for more information.

In [ ]:
client.agents.update(agent_id=agent_id, llm_model_id=tune_metadata.model_id)

print("Agent updated with tuned model")

## Next Steps

In this Notebook, we've created a RAG agent in the finance domain, evaluating the agent, and tuned it for better performance. You can learn more at [docs.contextual.ai](https://docs.contextual.ai/). Finally, reach out to your account team if you have further questions or issues.